# Imports and Pre-requisites

In [1]:
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, timedelta, datetime

In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize(project="rwanda-climate-alerts")

# Collect Datasets

## Baseline setup
In this section, we will define the
- baseline area of interest: polygon covering Rwanda
- drought/flood years

In [8]:
# drought_years = [
#     "1984-10-01",
#     "1989-12-01",
#     "1996-01-01",
#     "1999-11-01",
#     "2000-01-01",  # "Early 2000"
#     "2003-03-01",
#     "2005-02-01",
#     "2006-03-01",
#     "2006-09-01",
#     "2014-06-01"
# ]
#
# # Flood dates
# flood_years = [
#     "1988-05-06",
#     "2000-11-21",
#     "2001-09-22",
#     "2001-10-30",
#     "2002-04-26",
#     "2003-10-30",
#     "2005-08-16"
# ]

In [71]:
# Convert date string to date format
def str_to_date_range(date_string, date_format):
    initial_date = datetime.strptime(date_string, date_format)

    two_weeks_early_date = initial_date - timedelta(weeks=2)
    two_weeks_later_date = initial_date + timedelta(weeks=2)

    return two_weeks_early_date, two_weeks_later_date

# date_1, date_2 = str_to_date_range("1970-01-01", "%Y-%m-%d")
# print(date_1, date_2)

def date_range_to_list(date_list):
    date_range_list = []

    for flood_date in date_list:
        date_range_0, date_range_1 = str_to_date_range(flood_date, "%Y-%m-%d")
        date_range_list.append([date_range_0, date_range_1])

    return date_range_list

flood_dates_range = []

for flood_range in flood_dates_range:
    print(flood_range[0], flood_range[1])

In [72]:
aoi = ee.Geometry.Polygon([
    [
        [28.70, -2.85],
        [31.00, -2.85],
        [31.00, -1.00],
        [28.70, -1.00]
    ]
])

# Drought dates
drought_years = ["1984", "1989", "1996", "1999", "2000", "2003", "2005", "2006", "2006", "2014"]

# Flood dates
flood_years = ["1988", "2000", "2001", "2001", "2002", "2003", "2005"]

## Fetch Datasets

In [78]:
chirps_baseline_start = '1981-01-01'
chirps_baseline_end   = '2025-09-26'

# CHIRPS Daily Rainfall (mm/day)
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')\
            .filterDate("2000-01-01", "2025-08-01")\
            .filterBounds(aoi)

# ERA5 Land Monthly Temperature
era5_temp = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR") \
               .select("temperature_2m") \
               .filterDate("2000-01-01", "2025-08-01") \
               .filterBounds(aoi)

# Soil Moisture (ERA5 Land)
soil_moist = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR") \
                 .select("volumetric_soil_water_layer_1") \
                 .filterDate("2000-01-01", "2025-08-01") \
                 .filterBounds(aoi)

# annual_rain = chirps.sum()
# annual_rain_aoi = annual_rain.clip(aoi)

# event_year = "2024"
# event = chirps.filterDate(f"{event_year}-01-01", f"{event_year}-12-31")

## Harmonize the datasets

In [82]:
# Pick a reference projection (CHIRPS here)
ref = chirps.first().projection()

def harmonize(img):
    return img.resample("bilinear").reproject(ref)

chirps_h = chirps.map(harmonize)
era5_temp_h = era5_temp.map(harmonize)
soil_moist_h = soil_moist.map(harmonize)


## Compute pixel-level hazard indices

In [84]:
# Example: Monthly rainfall anomaly
monthly_rain = chirps_h\
    .map(lambda img: img.set("month", img.date().get("month")))

baseline_mean = monthly_rain.mean()

In [70]:
m = geemap.Map(center=[-1.94, 29.87], zoom=8)

m.addLayer(aoi, {'color': 'black'}, 'Geometry [black]: Rwanda')

m.addLayer(annual_rain_aoi, vis_params, 'CHIRPS')
palette = ['red', 'orange', 'yellow', 'green', 'blue', 'darkblue']
vis_params = {
    'min': 0,
    'max': 200000, # Adjust max based on expected precipitation range (in mm)
    'palette': palette
}

m.add_colorbar(vis_params, label="Rainfall (mm)")

m

Map(center=[-1.94, 29.87], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…